In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras_tuner import RandomSearch
import numpy as np

# ResNet 블록 정의
def resnet_block(x, filters, kernel_size=3, stride=1, conv_shortcut=True):
    shortcut = x

    if conv_shortcut:
        shortcut = layers.Conv2D(filters, 1, strides=stride)(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Conv2D(filters, kernel_size, strides=stride, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv2D(filters, kernel_size, padding='same')(x)
    x = layers.BatchNormalization()(x)

    x = layers.Add()([shortcut, x])
    x = layers.Activation('relu')(x)

    return x

# 하이퍼파라미터 튜닝을 위한 모델 빌더
def build_model(hp):
    inputs = keras.Input(shape=(32, 32, 3))

    # 초기 컨볼루션 레이어
    x = layers.Conv2D(
        hp.Int('initial_filters', 32, 64, step=16),  # 1. Unit size 튜닝
        kernel_size=hp.Choice('initial_kernel', [3, 5, 7]),  # 2. Kernel size 튜닝
        strides=2,
        padding='same'
    )(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

    # ResNet 블록
    for i in range(hp.Int('n_blocks', 2, 4)):  # 3. 레이어 수 튜닝
        x = resnet_block(x, 64 * (2 ** i))

    # 글로벌 풀링
    x = layers.GlobalAveragePooling2D()(x)

    # Dropout
    dropout_rate = hp.Float('dropout', 0.2, 0.5, step=0.1)  # 4. Dropout rate 튜닝
    x = layers.Dropout(dropout_rate)(x)

    # 출력 레이어
    outputs = layers.Dense(10, activation='softmax')(x)

    model = keras.Model(inputs, outputs)

    # 옵티마이저 및 학습률 튜닝
    learning_rate = hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')  # 5. Learning rate 튜닝
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# 데이터 로드 및 전처리
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# 검증 세트 분리
val_size = 5000
x_val = x_train[-val_size:]
y_val = y_train[-val_size:]
x_train = x_train[:-val_size]
y_train = y_train[:-val_size]

# GPU 확인
print("GPU Available: ", tf.config.list_physical_devices('GPU'))

# 하이퍼파라미터 튜닝 실행
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=10,
    directory='cifar10_resnet_tuning',
    project_name='cifar10_resnet'
)

tuner.search(
    x_train,
    y_train,
    epochs=5,
    batch_size=64,
    validation_data=(x_val, y_val),
    callbacks=[keras.callbacks.EarlyStopping(patience=3)]
)

# 최적의 하이퍼파라미터 출력
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("\n최적의 하이퍼파라미터:")
print(f"Initial filters: {best_hps.get('initial_filters')}")
print(f"Initial kernel size: {best_hps.get('initial_kernel')}")
print(f"Number of ResNet blocks: {best_hps.get('n_blocks')}")
print(f"Dropout rate: {best_hps.get('dropout')}")
print(f"Learning rate: {best_hps.get('learning_rate')}")

# 최적의 모델로 최종 평가
best_model = tuner.get_best_models(num_models=1)[0]
evaluation = best_model.evaluate(x_test, y_test)
print(f"\n테스트 세트 성능:")
print(f"Loss: {evaluation[0]:.4f}")
print(f"Accuracy: {evaluation[1]:.4f}")

# 학습 곡선 시각화
best_model.fit(
    x_train,
    y_train,
    epochs=30,
    batch_size=64,
    validation_data=(x_val, y_val),
    callbacks=[keras.callbacks.EarlyStopping(patience=5)],
)


Trial 1 Complete [00h 52m 05s]
val_accuracy: 0.6567999720573425

Best val_accuracy So Far: 0.6567999720573425
Total elapsed time: 00h 52m 05s

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
48                |32                |initial_filters
7                 |5                 |initial_kernel
2                 |3                 |n_blocks
0.4               |0.2               |dropout
0.0020706         |0.00013571        |learning_rate

Epoch 1/5
704/704 ━━━━━━━━━━━━━━━━━━━━ 207s 285ms/step - accuracy: 0.3898 - loss: 1.6750 - val_accuracy: 0.5520 - val_loss: 1.2629
Epoch 2/5
356/704 ━━━━━━━━━━━━━━━━━━━━ 1:33 268ms/step - accuracy: 0.5771 - loss: 1.1767